# First Iteration
Now that I am done scraping (please dear god no more I dont want to for the time being), I'm going to do a clustering thingy thing!

In [159]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import numpy as np
import nltk
import os
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, TruncatedSVD

#from sklearn.model_selection import train_test_split

In [83]:
# loading in my dataset

song_df = pd.DataFrame(pd.read_pickle('third_round_results_lyrics.pkl')).reset_index()
song_df.drop('index',axis = 1, inplace = True, errors = 'ignore')
lyrics = song_df.drop(['artist', 'song'], axis = 1, errors = 'ignore')
lyrics.head(1)



,lyrics
0,The last leaf clings to the bough. Just one le...


In [111]:
lyrics.lyrics = lyrics.lyrics.apply(', '.join)

The interesting thing is that when you remove baby or like or love or honey, you only get 2 topics.... WHY

In [179]:
from sklearn.feature_extraction import text

my_stop_words = text.ENGLISH_STOP_WORDS.union(["na", 'instrumental', 'yeah', 'ha', 'ok', 'babe', 'oo'])

vectorizer = TfidfVectorizer(ngram_range=(1,1), stop_words=my_stop_words)

In [180]:
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words=my_stop_words, 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', 
                                   token_pattern="\\b[a-z][a-z]+\\b",
                                   lowercase=True,
                                   max_df = 0.6)

cv_data = count_vectorizer.fit_transform(list(lyrics.lyrics.values))
tfidf_data = tfidf_vectorizer.fit_transform(list(lyrics.lyrics.values))

In [181]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [189]:

# We do one of these for you (lsa_tfidf)
# Make the other two
n_comp = 50
lsa_tfidf = TruncatedSVD(n_components=n_comp)

lsa_tfidf_data = lsa_tfidf.fit_transform(tfidf_data)

display_topics(lsa_tfidf,count_vectorizer.get_feature_names(),5)


Topic  0
love conquer, don cause, baby bow, know darling, oh honey

Topic  1
love conquer, love remains, oh vivid, oh honey, just request

Topic  2
oh honey, oh vivid, baby bow, come da, yes hold

Topic  3
baby bow, baby coast, yes hold, zoooom, come dim

Topic  4
lady ll, lady takin, natural selection, naughty, yes hold

Topic  5
lady ll, don cause, lady takin, ll discover, natural selection

Topic  6
ll discover, come da, awake hours, heart falling, ll send

Topic  7
natural selection, naughty, ll discover, door bibles, sad crying

Topic  8
intoxicatin, like capital, feel cries, ll discover, come da

Topic  9
yes hold, zoooom, don cause, let feel, ll discover

Topic  10
let feel, come da, want light, like capital, got burning

Topic  11
want light, got burning, ll discover, know darling, need lonely

Topic  12
got burning, ve long, ve spiritual, let feel, hold note

Topic  13
come da, ve long, feel cries, come feelings, want light

Topic  14
want light, awake hours, gonna everybody,

In [190]:

# We do one of these for you (lsa_tfidf)
# Make the other two
n_comp = 25
lsa_tfidf = TruncatedSVD(n_components=n_comp)

lsa_tfidf_data = lsa_tfidf.fit_transform(tfidf_data)

display_topics(lsa_tfidf,count_vectorizer.get_feature_names(),5)


Topic  0
love conquer, don cause, baby bow, know darling, oh honey

Topic  1
love conquer, love remains, oh vivid, oh honey, just request

Topic  2
oh honey, oh vivid, baby bow, come da, yes hold

Topic  3
baby bow, baby coast, yes hold, zoooom, come dim

Topic  4
lady ll, lady takin, natural selection, naughty, yes hold

Topic  5
lady ll, don cause, lady takin, ll discover, natural selection

Topic  6
ll discover, come da, awake hours, ll send, heart falling

Topic  7
natural selection, naughty, ll discover, door bibles, sad crying

Topic  8
intoxicatin, like capital, heart falling, gonna everybody, awake hours

Topic  9
yes hold, zoooom, don cause, let feel, ll discover

Topic  10
let feel, come da, want light, like capital, got burning

Topic  11
want light, ll discover, got burning, know darling, need lonely

Topic  12
got burning, ve long, ve spiritual, let feel, hold note

Topic  13
come da, ve long, feel cries, come feelings, want light

Topic  14
want light, awake hours, gonna

In [186]:
# This displays the topics for using lsa on TF-IDF
# Get the top 10 for lsa using CountVecotizer
# Get the top 10 for NMF using CountVectorizer

n_comp = 20
lsa_cv = TruncatedSVD(n_components=n_comp)

lsa_cv_data = lsa_cv.fit_transform(cv_data)

display_topics(lsa_cv,count_vectorizer.get_feature_names(),10)


Topic  0
love, oh, baby, don, know, just, like, got, come, let

Topic  1
la, la la, ohh, la ohh, nigga, nothin, ohh la, nigga nothin, yo, like

Topic  2
like, don, rock, know, party, party like, like rock, got, make, just

Topic  3
jump, jump jump, oh, like, party, rock, oh oh, party like, like rock, come

Topic  4
baby, come, let, ding, come baby, baby baby, baby come, ding ding, ll, bounce

Topic  5
come, like, baby, oh, party, party like, oh oh, like rock, rock, star

Topic  6
love, jump, baby, jump jump, love love, baby baby, party, party like, come, like

Topic  7
walk, walk walk, northside, southside, westside, southside walk, westside walk, walk eastside, northside walk, eastside

Topic  8
step, wit, step step, jiggy, jiggy wit, gon, gon step, don, wit jiggy, step gon

Topic  9
don, stop, don stop, rock, party like, stop don, party, force, stop til, force don

Topic  10
cool, cool cool, round, said, chill, ve, cool said, chill cool, round round, don

Topic  11
make, dance, duh,